# InteractiveActivations

This notebook lets you load and explore character level activations from our models. Our analysis is based on aggregating unit activations over many words, but it is interesting to see how the network processes words.

In [1]:
%load_ext autoreload

%autoreload 2
import numpy as np
import dynet as dy
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib inline

from infer import WrappedTagger
from utils import Instance
from analysis_utils import get_best_model_name 

import seaborn as sns
sns.set()

### Datasets/Models

Change `ln` to load different models. 

In [2]:
ln = 'en'
ln = 'tr'

model = WrappedTagger('datasets/ud-2.3/{}_ud23.pkl'.format(ln),
                      num_lstm_layers=2,
                      hidden_dim=128,
                      char_embedding_dim=256,
                      word_level_dim=128)
model.load_weights(get_best_model_name('models/{}/'.format(ln)))

INFO:root:char bilstm: char_embedding_dim 256 hidden 128
INFO:root:word bilstm: input_dim 128 word_level_dim 128


models/tr/
Using legacy loading


### Available Alphabet 

All of the characters in the character level vocabulary. 

In [3]:
print(' '.join(sorted(model.dataset.c2i.keys())))

! " ' ( ) , - . 0 1 2 3 4 5 6 7 8 9 : ; <*> ? A B C D E F G H I J K L M N O P R S T U V W Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z Â Ç Ö Ü ç ö ü Ğ ğ İ ı Ş ş


### Interactive 

Interactively plot activations for a word or (tokenized) sentence. The slider controls which dimension of the hidden state (unit) is shown. We also display the predicted part-of-speech tag. The part of speech tags can incorporate information from other words in the sentence. 

In [4]:
def show_full(sentence, dim=0):
    f, a = plt.subplots(1, figsize=(10, 10))
    out_tags, embeddings = model.forward_text(sentence)
    print(model.get_pos(out_tags))
    activations = []
    for word, word_embedding in zip(sentence.split(' '), embeddings):
        for char, char_embedding in zip('*' + word + '*', word_embedding):
            if char == '*':
                activations.append(np.nan)
            else:
                activations.append(char_embedding.value()[dim])
                
    a.plot([0, len(activations)], [0.0, 0.0], color='black')
    a.plot(activations, ls='dotted', linewidth=6)
    
    a.set_xticks(range(len(activations)))
    a.set_xticklabels(' ' + '  '.join(sentence.split(' ')) + ' ')
    a.xaxis.set_ticks_position('none')
    
    a.set_ylabel('activation', fontdict={'fontsize':30})
    a.set_yticks(np.arange(-1.0, 1.5, 0.5))
    plt.yticks(fontsize=30)
    plt.xticks(fontsize=30)

In [5]:
# ev-ler-iniz-den 
# see https://en.wikipedia.org/wiki/Agglutination :) 

word_input = widgets.Text(value='evlerinizden', continuous_update=False, description='Input')
# word_input = widgets.Text(value='This is a wug .', continuous_update=False, description='Input')

dim_input = widgets.IntSlider(min = 0, max = 127, step = 1, value = 0, continuous_update=False)
plot = interactive(show_full, sentence=word_input, dim=dim_input)
output = plot.children[-1]
output.layout.height = '700px'
plot

interactive(children=(Text(value='evlerinizden', continuous_update=False, description='Input'), IntSlider(valu…